In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pycountry -q
!pip install pycountry_convert -q

In [ ]:
import warnings;   warnings.filterwarnings("ignore")

# Wizualizacja Danych

EDA jest skrótem od **E**xploratory **D**ata **A**nalysis i EDA jest kluczowym etapem poprzedzającym stosowanie modelu ML. Jak sama nazwa wskazuje, chodzi o eksplorację danych — sprawdzenie, czy zestaw danych, nad którym będziesz pracować, jest spójny i nie zawiera brakujących wartości. Ważnym elementem jest także wizualizacja samuych danych aby lepiej przyjrzeć się ich dystrybucji (rozmieszczeniu) i zachowdzącym relacjom. 

Warto przy tym zauważyć, że nie każdy problem wymaga zastosowania uczenia maszynowego. Być może w pracy zadaniem będzie wizualizacja danych w celu analizy i przedstawienia najciekawszych spostrzeżeń, które cięzko by było zauważyć patrząć na suche dane w tabeli.

Przy formułowaniu problemu ważne jest postawienie hipotezy. To pomoże nam lepiej wybrać dane do analizy.
Aby wyjaśnić, dlaczego jest to konieczne, poniższy obraz przedstawia coś znanego jako *Kwartet Ascomebe'a*:

<img src="https://www.researchgate.net/profile/Arch_Woodside2/publication/285672900/figure/fig4/AS:305089983074309@1449750528742/Anscombes-quartet-of-different-XY-plots-of-four-data-sets-having-identical-averages.png">

Źródło: https://www.researchgate.net/publication/285672900_The_general_theory_of_culture_entrepreneurship_innovation_and_quality-of-life_Comparing_nurturing_versus_thwarting_enterprise_start-ups_in_BRIC_Denmark_Germany_and_the_United_States

Kwartet Ascomebe'a jest przykładem, dlaczego wizualizacja naszych danych ma ogromne znaczenie. Obraz pokazuje nam, że statystyki podsumowujące (np. średnia, wariancja) dla wszystkich danych są takie same. Jednak, jak widać, rozkłady, z których pochodzą dane, są szalenie różne. Gdybyśmy nie wizualizowali naszych danych, nie bylibyśmy w stanie w prosty sposób zidentyfikować zależności między danymi.

W tym notatniku przedstawiam uproszczony proces EDA w celach dydaktycznych. 
W rzeczywistości czyszczenie i usuwanie brakujących danych podlega parasolowi EDA i będzie to cykliczny proces, w którym eksplorujesz swoje dane, identyfikujesz probloemy, czyścisz je i ponownie eksplorujesz.


Będziemy pracować z plikiem „multiple_choice_responses.csv” z [2019 Kaggle ML & DS Survey](https://www.kaggle.com/c/kaggle-survey-2019/data?select=multiple_choice_responses.csv), ktÓry zawiera 35 pytań przeprowadzonych na użytkownikach Kaggle, dotyczących stanu nauki o danych i uczenia maszynowego. Według abstraktu, ankieta ta otrzymała 19 717 użytecznych respondentów ze 171 krajów i terytoriów. Jeśli kraj lub terytorium otrzymało mniej niż 50 respondentów, pogrupowaliśmy ich w grupę o nazwie „Inne” w celu zachowania anonimowości. Zadaniem, które stoi przed nami jest określenie, jakie czynniki mają istotny wpływ na roczną pensję osób w DSML (Data Science i Machine Learning).



In [ ]:
## Załaduj dane i wypisz pierwsze wiersze
import pandas as pd
pd.options.display.max_columns = None

path = './drive/MyDrive/Nauczanie/202223/PAD/05_wizualizacja plotly'
df = pd.read_csv(f"{path}/multiple_choice_responses.csv")
df.head(3)

In [ ]:
## Mamy też plik "questions_only.csv". Załaduj go i wczytaj wszystkie pytania
q_df = pd.read_csv(f"{path}/questions_only.csv")
for i, question in enumerate(q_df.iloc[0]):

    print(i, "\t", question)

Wstępnie możemy zauważyć, że:
- Jest wiele pytań (dużo danych do przeanalizowania)
- Niektóre pytania pozwalają na wiele danych wejściowych (pytania wielokrotnego wyboru). W przypadku tych pytań nazwy wierszy/kolumn nagłówka mają dodany znak `_`, po którym następuje jakiś tekst.
  - Jeśli tekst to `OTHER_TEXT`, to wydaje się wskazywać, że po kategorycznym pytaniu wyświetlane jest pole tekstowe z opcją rozwinięcia dla odbiorcy. Wygląda na to, że -1 oznacza, że użytkownik nic nie napisał.
  - Jeśli tekt to `PART_N`, wówczas wydaje się to być pytaniem wielokrotnego wyboru do zaznaczenia. 
  - Nie wykluczają się wzajemnie

Analiza tych danych kolumna po kolumnie zajmie zbyt wiele czasu. Zamierzamy więc zdecydować o czynnikach, które naszym zdaniem mogą mieć wpływ na wynagrodzenie, i wyodrębnimy z listy odpowiednie pytania, które spełniają te kryteria. Częściowo dlatego nauka o danych jest uważana za sztukę – możesz mieć do czynienia z ogromnym zbiorem danych i nie mieć pewności, od czego zacząć analizę. Korzystając ze swojej hipotezy i identyfikując, co próbujesz modelować, musisz użyć swojej intuicji, co do czynników, które Twoim zdaniem będą miały na to duży wpływ. Dlatego ważna jest znajomość dziedziny. Ale im więcej eksplorujesz swoje dane z początkowymi koncepcjami, które miałeś na myśli, tym więcej w końcu dowiesz się o szerszym zbiorze danych.
  
- Salary (target)
- Age
- Gender
- Residence
- Education
- Job role/Experience
- Programming languages
- ML frameworks

Z tej listy wyodrębnimy następujące pytania:
**Q:** 1, 2, 3, 4, 5, 9, **10**, 15, 18, 24, 28.

Jest  kilka innych, które również byłyby istotne do analizy - w idealnym świecie też byśmy je analizowali, ale tutaj czas jest ograniczony - i ważne jest, aby nauczyć Cię różnych technik wizualizacyjnych, jednocześnie budując intuicję, na co patrzeć w danych.

Niektóre z tych pytań obejmują wiele kolumn w naszej ramce danych. Wyodrębnianie odpowiednich kolumn, które chcemy, nie jest najprostszym zadaniem. Poświęć trochę czasu, aby spróbować zaimplementować coś, co zwraca nową ramkę danych zawierającą odpowiednie kolumny. Jeśli po kilku minutach nie masz pewności, jak postępować, kliknij poniżej, aby spróbować zaimplementować metodę, której ja bym użyła.


<details>
    <summary><b>Kliknij tutaj, aby zobaczyć co bym zrobiła</b></summary>
    <ul>
        <li>Zdefiniuj funkcję, która iteruje po liście integerów odpowiadającyh pytaniom, które chcemy zachować</li>
        <li>Dla każdej iteracji oblicz ilość kolumn od bieżącego pytania do następnego pytania w ramce danych (NIE następnego pytania, które chcemy wyodrębnić)</li>
        <li>Wyodrębnij/konkatenuj z bieżącej pozycji kolumny do bieżącej pozycji kolumny + 'odległość' (możesz skorzystać z funkcji <code>range()</code>)</li> 
    </ul>
</details>


#### rozwiazanie

In [ ]:
idx_to_keep = [1,2,3,4,5,9,10,15,18,24,28]

def extract_columns(df, idx_to_keep):
    
    new_df = pd.DataFrame() # pusty dataframe
    df_col_list = df.columns.tolist()
    
    for i in idx_to_keep:
        column_name_base = "Q{}".format(i)
        column_index = [df_col_list.index(col_name) for col_name in df_col_list if col_name.startswith(column_name_base)][0]
               
        next_column_name_base = "Q{}".format(i+1)
        next_column_index = [df_col_list.index(col_name) for col_name in df_col_list if col_name.startswith(next_column_name_base)][0]
         
        col_idxs_to_extract = range(column_index, next_column_index)
        relevant_cols_df = df.iloc[:, col_idxs_to_extract]
        
        new_df = pd.concat([new_df, relevant_cols_df], axis=1)
        
    return new_df


df_orig = df.copy(deep=True)
df = extract_columns(df_orig, idx_to_keep)
df = df[1:]
df.head()

Wciąż... dużo danych... cóż, od czegoś musimy zacząć! Arbitralnie zacznijmy od Gender (Q2). Widzimy, że dane tutaj mają być kategoryczne, więc po upewnieniu się, że tak jest, wykreślmy po prostu częstotliwość każdej z wartości

In [ ]:
df["Q2"] = df["Q2"].astype("category")
set(df["Q2"])

### HISTOGRAM: 
Korzystając z biblioteki plotly narysuj **HISTOGRAM** prezentujący rozkład płci.

In [ ]:
import plotly.express as px



Świetnie! A co z miejscami zamieszkania osób? Ulepszymy wykres i wykreślimy je na mapie świata, uzupełniając o liczbą respondentów z tego kraju (w skali temperatury). Jest to znane jako [mapa choropletowa](https://plotly.com/python/choropleth-maps/), innymi słowy kartogram, i wymaga od nas zmiany nazw naszych krajów na [trzyliterowe kody ISO](https://pl.wikipedia.org/wiki/ISO_3166-1_alfa-3).

Pierwszą rzeczą, którą musimy zrobić, to spojrzeć na samą kolumnę krajów (tj. Q3). Po wykonaniu tej czynności warto zaktualizować wartości do bardziej konwencjonalnych, jeśli jeszcze ich nie ma.

Następnie załadujemy paczkę, do której będziemy mogli przekazać kraj i zwrócimy nam kod ISO. Następnie użyjemy nowej kolumny do wykreślenia naszego kartogramu.

In [ ]:
# warto przyjżeć się wartościom i zdecydować czy nie trzeba czegoś poprawić
set(df["Q3"])

Oto wartości, które moim zdaniem wymagają aktualizacji:
- Hong Kong (S.A.R.)
- Iran, Islamic Republic of...
- United Kingdom of Great Britain and Northern Ireland
- Viet Nam
- South Korea

Zauważ, że występują również "Other"

In [ ]:
print("Percentage of 'Other':", df["Q3"].value_counts()["Other"]/len(df) * 100)

values_to_update = {"Q3": 
                    {"Hong Kong (S.A.R.)": "Hong Kong",
                     "Iran, Islamic Republic of...": "Iran",
                     "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
                     "South Korea": "Republic of Korea",
                     "Viet Nam": "Vietnam"}}

## Korzystając z metody replace, zaktualizuj wartości w odpowiedniej kolumnie
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.replace.html
df.replace(values_to_update, inplace=True)
set(df["Q3"])

In [ ]:
import pycountry

## Stwórz nowy dataframe, który będzie zawierał tylko unikalne wartości countries, ich kody oraz liczbę instancji danego kraju - BEZ "Other"
countries = df["Q3"][df["Q3"]!= "Other"].unique()
countries_df = pd.DataFrame(countries, columns=["Country"])
countries_df["Count"] = countries_df["Country"].map(df["Q3"].value_counts())

## Stwórz nową kolumnę, która będzie zawierała kod ISO
country_codes = []
for country in countries_df["Country"]:
    country_code = pycountry.countries.search_fuzzy(country)[0] # Wybierz pierwszy element zwracany przez search (pamiętaj o tym co oznacza pierwszy element dla programisty - index!)
    country_codes.append(country_code.alpha_3)

countries_df["Country Code"] = country_codes
countries_df.head()

In [ ]:
# mapa

A co ze zmienną gender? Trzeba zgrupować zmienne razem.

### WYKRES SŁUPKOWY: 
Korzystając z biblioteki plotly narysuj wykres słupkowy prezentujący rozkład płci ale zgrupowany. Dodaj odpowiednie nazwy etykietom: "Q1": "Age", "value": "Count"

In [ ]:
age_gender_df = df[["Q1", "Q2"]]
age_gender_groups = age_gender_df.groupby(["Q1", "Q2"]).size().unstack()



fig.show()

Widzimy więc, że najczęstszy wiek pracowników DSML to 25-29 lat. Widzę dwa powody, dla których te wartości są znacznie wyższe od pozostałych:
1. Data Science i Machine Learning to stosunkowo nowa dyscyplina, a teraz istnieją bezpośrednie ścieżki edukacyjne do tych dziedzin, które są bardziej dostępne dla młodszych ludzi.
2. Zastanów się, _skąd_ zebrano dane. Starsi ludzie są prawdopodobnie mniej skłonni do korzystania z witryn „zasobowych”, takich jak Kaggle, ponieważ 1) nie czują potrzeby uczenia się i 2) młodsi ludzie częściej pojawiają się w poertalach społecznościowych.

Do tej pory po prostu tworzyliśmy wykresy – być może lepszym planem jest przeprowadzenie nieco bardziej dochodzeniowej analizy kategorii, które wyznaczyliśmy wcześniej. Zróbmy to z Edukacją.

### ZADANIE 1 (4 pkt):
Stwórz dwa wykresy:
1. Formalne wykształcenie uczestników (histogram) - kolumna Q4, zamień etykietę label na Education Level, tytuł może być Count of Education
2. Formalne wykształcenie według płci. Wyświetl to jako zgrupowany wykres słupkowy (wykres słupkowy)

In [ ]:

fig.show()

Stwórzmy kolejny diagram przedstawiający te same informacje, ale na 4 różnych wykresach (kolumna Q2 - gender ma być przedstawiona różnymi kolorami)

In [ ]:

# fig.update_xaxes(showticklabels=True)
fig.show()

Kilka interesujących obserwacji:
1. Ci, którzy zdecydują się na podanie swojej płci opisowo, częściej mają stopień magistra niż licencjata – w porównaniu z każdą inną kategorią, która częściej ma stopień licencjata niż doktora. Chociaż jeśli zanotujemy liczby, zobaczymy, że pracujemy z liczbami jednocyfrowymi - nie jest to coś, co możemy ekstrapolować statystycznie (przewidzieć przebieg danego zjawiska w warunkach nieznanych na podstawie znajomości analogicznego zjawiska w znanych warunkach).
2. Ci, którzy woleli nie podawać swojej płci, woleli również nie podawać swojego poziomu wykształcenia (w stosunku do pozostałych kategorii).

Koloejny wykres jest znany jako [Diagram Sankeya](https://en.wikipedia.org/wiki/Sankey_diagram).

Najłatwiejszym sposobem rozpoczęcia pracy z diagramem Sankeya jest zrozumienie, czego oczekujemy jako kolumny końcowej. W tym przypadku jako ostatnią kolumnę użyjemy poziomu wykształcenia. Będziemy także potrzebować obliczyć ile osób jest na każdym poziomie edukacji. Aby zaoszczędzić miejsce na diagramie, uogólnimy niektóre poziomy.

Na tym diagramie Sankeya chcę zobrazować drogę płci, wieku i kraju do poziomu wykształcenia.

In [ ]:
# Chcemy pięciu poziomów edukacji: Bachelor's, Master's, Doctoral, Professional, Other
## Utwórz nową ramkę danych z wykształceniem ankietowanych, gdzie ich wykształcenie zostało zmapowane do powyższego poziomu
education_df = pd.DataFrame(df["Q4"])
education_df.rename(columns={"Q4": "Education Level"}, inplace=True)

values_to_update = {"Education Level": 
                    {"Some college/university study without earning a bachelor’s degree": "Other",
                     "No formal education past high school": "Other",
                     "I prefer not to answer": "Other"}}

education_df = education_df.replace(values_to_update)
set(education_df["Education Level"])

In [ ]:
# Usuńmy wartosci na z poziomu edukacji
education_df.isna().sum()
education_df = education_df.dropna(subset=["Education Level"])
education_df.isna().sum()

In [ ]:
## Do nowej ramki dodaj kolumny gender, age i region, nazywając je w odpowiedniej kolejności
cols_to_join = ["Q1", "Q2", "Q3"] 
desired_col_names = ["Age", "Gender", "Region"]
for col, name in zip(cols_to_join, desired_col_names):
    education_df[name] = df[col]
    
education_df

In [ ]:
# Dla celów wizualizacyjnych stwórzmy:
# 1. szersze przedziały wieku, jak 18-29, 30-49, 50-69 oraz 70+
# 2. grupy płci jako "Male", "Female", "Other"
# 3. zamieńmy kraje na kontynenty, poza: "India", "United States of America" oraz "Other"

values_to_update = {
    "Age": {"18-21": "18-29", "22-24": "18-29", "25-29": "18-29",
            "30-34": "30-49", "35-39": "30-49", "40-44": "30-49", "45-49": "30-49",
            "50-54": "50-69", "55-59": "50-69", "60-69": "50-69"
           },
    "Gender": {"Prefer not to say": "Other", "Prefer to self-describe": "Other"}
}

education_df = education_df.replace(values_to_update)
education_df

In [ ]:
import pycountry_convert as pc
## Mapuj kraje na odpowiednie kontynenty, chyba że krajem są Indie, Stany Zjednoczone Ameryki lub inne
countries_to_not_map = ["India", "United States of America", "Other"]
countries_to_map_to_continents = set(education_df["Region"])
for country in countries_to_not_map:
    countries_to_map_to_continents.discard(country)

countries_continent_dict = dict()
for country in countries_to_map_to_continents:
    country_alpha2 = pycountry.countries.search_fuzzy(country)[0].alpha_2
    continent_code = pc.country_alpha2_to_continent_code(country_alpha2)
    continent_name = pc.convert_continent_code_to_continent_name(continent_code)
    countries_continent_dict[country] = continent_name

to_update = {"Region": countries_continent_dict}
education_df = education_df.replace(to_update)
education_df

In [ ]:
# Ponowne indeksowanie kolumn w kolejności, w jakiej chcemy je umieścić na diagramie, ponieważ będzie łatwiej z nimi pracować
education_df = education_df.reindex(["Gender", "Age", "Region", "Education Level"], axis=1)

col_names = education_df.columns.tolist()
node_labels = []
num_categorical_vals_per_col = []
for col in col_names:
    uniques = education_df[col].unique().tolist()
    node_labels.extend(uniques)
    num_categorical_vals_per_col.append(len(uniques))
    
node_labels, num_categorical_vals_per_col

Ok, więc po obejrzeniu diagramu Sankeya wiemy, że poprzednia kolumna połączy się tylko z następną kolumną. Parametr `num_categorical_vals_per_col` pozwoli nam dowiedzieć się, które wartości z poprzedniej musimy zmapować na następne.

Teraz musimy skonstruować nasz słownik `link`. Jest to nieco mniej proste niż powyższe. Nasz słownik `link` będzie zawierał 3 listy: `source`, `target` i `value`. `source` i `target` wskazują, które węzły chcemy ze sobą połączyć, a `value` wskazuje ilość, którą chcemy „wypełnić” to połączenie. `source` i `target` to numeryczne indeksy listy `node_labels`, którą utworzyliśmy powyżej.

Dla każdej kategorii na kolumnę (kategoria źródłowa) połączymy tę kategorię ze wszystkimi innymi kategoriami następnej kolumny (kategoria docelowa), z rozmiarem, ile kategorii źródłowych jest mapowanych na kategorie docelowe.

In [ ]:
education_df.groupby(["Gender", "Age"]).size()["Female"]["18-29"]

In [ ]:
import numpy as np
import random

source = []
target = []
value = []
colors = []
for i, num_categories in enumerate(num_categorical_vals_per_col):
    
    if i == len(num_categorical_vals_per_col)-1:
        break
    
    # indeks umożliwia nam odwoływanie się do kategorii według indeksu z listy `node_labels`
    start_index = sum(num_categorical_vals_per_col[:i])
    start_index_next = sum(num_categorical_vals_per_col[:i+1])
    end_index_next = sum(num_categorical_vals_per_col[:i+2])
#     print(start_index, start_index_next, end_index_next)
    
    # może również dać nam kolumnę kategorii, do której się odwołujemy
    col_name = col_names[i]
    next_col_name = col_names[i+1]
    
    grouped_df = education_df.groupby([col_name, next_col_name]).size()
#     print(grouped_df)
    
    for source_i in range(start_index, start_index_next):
        for target_i in range(start_index_next, end_index_next):
            source.append(source_i)
            target.append(target_i)
            source_label = node_labels[source_i]
            target_label = node_labels[target_i]
            # jeśli indeks nie istnieje w grouped_df, to wartość wynosi 0
            try:
                value.append(grouped_df[source_label][target_label])
            except:
                value.append(0)
            
            random_color = list(np.random.randint(256, size=3)) + [random.random()]
            random_color_string = ','.join(map(str, random_color))
            colors.append('rgba({})'.format(random_color_string))

print(source)
print(target)
print(value)

link = dict(source=source, target=target, value=value, color=colors)

In [ ]:


fig.update_layout(title_text="Sankey Diagram (Gender, Age, Region, Education)", font_size=10)
fig.show()

***Wiek***

Wcześniej bardzo szybko przyjrzeliśmy się wiekowi, ale chciałabym dowiedzieć się więcej na jego temat. Używając oryginalnego `df`, utwórz następujące wykresy:
1. Wykres fasetowy liczby poziomów wykształcenia według wieku
2. Aspekty różnych ról według wieku

Następnie omówimy następujące kwestie:
1. Wykres liczby różnych języków
2. Wykresy podrzędne/wykres fasetowy liczby różnych języków na wiek

### ZADANIE 2 (2 pkt): 
Korzystając z biblioteki plotly narysuj **WYKRES FASETOWY** prezentujący  liczbę poziomów wykształcenia wg. wieku

Na każdym poziomie mają być widoczne 4 wykresy, wiek to kolumna Q1.

In [ ]:
# Posortujemy plik df według wieku, aby nasz wykres wyświetlał się w kolejności wiekowej
df = df.sort_values(by=["Q1"])


Ciekawe! Być może wyniki są tym, czego byśmy się spodziewali - osoby w wieku 18-21 lat są zazwyczaj za młode, aby uzyskać tytuły magisterskie, stąd liczba licencjatów jest dla nich wyższa. Jednak w prawie każdej innej grupie wiekowej wyróżniają się stopnie magisterskie. Co ciekawe, osoby powyżej 70. roku życia częściej mają doktorat 😀

In [ ]:
set(df["Q5"])

In [ ]:
fig = px.histogram(df, "Q5", 
             facet_col="Q1",
             color="Q1",
             title="Counts of Education level per Age",
             labels={"Q1": "Age", "Q5": "Job Role"},
             height=2000, 
             facet_col_wrap=2, 
             facet_col_spacing=0.1)

fig.update_layout(showlegend=False)
fig.update_xaxes(showticklabels=True, tickangle=45)
fig.update_yaxes(matches=None, showticklabels=True)

In [ ]:
# Nasze dane znajdują się w kolumnach 18_p1, 18_p12
# Naszym pierwszym krokiem będzie utworzenie nowej kolumny o nazwie „Known programming languages” i utworzenie dla każdego wiersza listy oddzielonej przecinkami zawierającej języki programowania, które znają (oczywiście z wyłączeniem NaNs)
programming_cols = ["Q18_Part_{}".format(str(i)) for i in range(1, 13)]
programming_df = df[programming_cols]
programming_df.head(5)

In [ ]:
programming_col = []
for row in programming_df.itertuples(index=False):
    languages_known = [language for language in row if isinstance(language, str)]
    programming_col.append(",".join(languages_known))
    
programming_df["languages_known"] = programming_col
programming_df

In [ ]:
# Przytnijmy nowy df, żeby miał tylko nasz nowy col
programming_df.drop(labels=programming_cols, axis=1, inplace=True)
programming_df

In [ ]:
# Załóż, że spacje oznaczają, że nie znają języka i zamień zarówno spacje, jak i „None” na „None/NA”
values_to_update = {"languages_known": {"": "None/NA", "None": "None/NA"}}
programming_df = programming_df.replace(values_to_update)
programming_df

In [ ]:
# Użyjmy metody get_dummies do stworzenia nowych kolumn
language_dummies = programming_df['languages_known'].str.get_dummies(sep=',')
language_dummies

In [ ]:
fig = px.bar(language_dummies.sum(), labels={"index": "Programming Language"}, title="Count of Programming Languages")
fig.update_layout(showlegend=False, xaxis={'categoryorder':'total descending'})
fig.show()

In [ ]:
# Dla 4. pobierzmy wiek z oryginalnej ramki danych i połączmy je z tą nową ramką danych
ages = df["Q1"]
language_dummies_with_age = language_dummies.join(ages).rename(columns={"Q1": "Age"})
language_dummies_with_age

In [ ]:
programming_languages_by_age = language_dummies_with_age.groupby(["Age"]).sum()
px.bar(programming_languages_by_age)
# px.bar(programming_languages_by_age.T)
programming_languages_by_age = programming_languages_by_age.reindex(
    programming_languages_by_age.mean().sort_values().index, axis=1)
programming_languages_by_age = programming_languages_by_age.iloc[:, ::-1]
programming_languages_by_age

In [ ]:
programming_languages_by_age_row_norm = programming_languages_by_age.div(programming_languages_by_age.sum(axis=1), axis=0)

Stwórzmy wykres z podwykresami, który będzie pokazywał Percent of Known Programming Languages by Age

In [ ]:
from plotly.subplots import make_subplots

# programming_languages_by_age.index
programming_languages = programming_languages_by_age_row_norm.columns.tolist()



fig.update_layout(showlegend=False, height=1000, title="Percent of Known Programming Languages by Age")
fig.update_yaxes(tickformat="%")
fig.show()

Wygląda na to, że wszyscy lubią Pythona! Młodsi ludzie (którzy najprawdopodobniej robią licencjat) mają stosunkowo wyższy odsetek języków niższego poziomu, takich jak C, C++ i Java. Może to być spowodowane tym, że muszą uczyć się tych języków na uniwersytecie. Ponieważ naukowcy zajmujący się danymi bardziej specjalizują się w swojej karierze, wydają się odchodzić od tych języków do bardziej typowych języków związanych z DSML. Grupa 60-69 lat ma wysoki poziom użytkowników języka R w porównaniu z innymi grupami wiekowymi, podczas gdy wydaje się, że większość osób powyżej 70. roku życia nie zna żadnych języków programowania.

Być może bardziej przydatne dla nas jest to, jakie języki programowania są popularne w stosunku do zawodu (P5). Stwórz wykres, który to pokazuje

In [ ]:
## Połącz pytanie o zawód z ramką language_dummies
language_dummies_with_job = language_dummies.join(df["Q5"]).rename(columns={"Q5": "Job Title"})
language_dummies_with_job

In [ ]:
## Pogrupuj według stanowiska i zagreguj liczbę, znormalizuj każdy wiersz i posortuj ramkę danych według średniej z kolumn
languages_job_title_grouped = language_dummies_with_job.groupby(["Job Title"]).sum()
languages_job_title_grouped = languages_job_title_grouped.div(languages_job_title_grouped.sum(axis=1), axis=0)
languages_job_title_grouped = languages_job_title_grouped.reindex(
    languages_job_title_grouped.mean().sort_values().index, axis=1)
languages_job_title_grouped = languages_job_title_grouped.iloc[:, ::-1]
languages_job_title_grouped

### ZADANIE 3 (1 pkt):
Stwórz wykres mapy cieplnej powiązania języków programowania i nazwy zawodu (job title)

### ZADANIE 4 (4 pkt):
Stwórz wykres z podwykresami, który będzie pokazywał Known Programming Language per Job

In [ ]:
## Stwórzy wykres danych!
programming_languages = languages_job_title_grouped.columns.tolist()


fig.show()

**Obserwacje**:
- Python jest średnio najpopularniejszym językiem spośród wszystkich stanowisk pracy
- SQL jest najpopularniejszym językiem wśród inżynierów baz danych
- MATLAB jest stosunkowo bardziej popularny wśród naukowców niż na innych stanowiskach pracy
- Statystyk woli R od Pythona
- Studenci i inżynierowie oprogramowania wolą C++ od R

Zróbmy jeszcze jeden wykres - mapę cieplną frameworka, z którego lubi korzystać każda rola zawodowa

In [ ]:
## Wykreśl 3 najlepsze frameworki, z których każda rola lubi korzystać
# 28p1 - 28p12
## Utwórz ramkę danych, która zawiera tylko kolumnę z frameworkiem
framework_cols = ["Q28_Part_{}".format(str(i)) for i in range(1, 13)]
framework_df = df[framework_cols]
framework_df

Utwórz ogólną funkcję `get_df_for_dummies()`, która pobiera ramkę danych, ciąg przedrostka nazwy kolumny oraz górny zakres i zwraca ramkę danych wypełnioną zakresem kolumn na podstawie ciągu przedrostka

In [ ]:
def get_df_for_dummies(df, prefix_string, end_range, start_range=1):
    ## WSKAZÓWKA: skorzystaj z funckji range oraz iteracji aby wygenerować nazwy kolumn
    dummies_cols = [prefix_string + str(i) for i in range(start_range, end_range)]
    ## WSKAZÓWKA: wyciągnij i zwróć nazwy kolumn z ramki danych
    return df[dummies_cols]

In [ ]:
## Utwórz kolumnę w tej ramce danych o nazwie „Użyte struktury” i wypełnij tę kolumnę ramami oddzielonymi przecinkami
framework_col = []
for row in framework_df.itertuples(index=False):
    frameworks_used = [framework for framework in row if isinstance(framework, str)]
    framework_col.append(",".join(frameworks_used))
    
framework_df["frameworks_used"] = framework_col
framework_df

In [ ]:
## Zastąp kolumny puste i None wartościami "None/NA"
values_to_update = {"frameworks_used": {"": "None/NA", "None": "None/NA"}}
framework_df = framework_df.replace(values_to_update)
framework_df

Utwórz ogólną funkcję `get_dummies()`, która pobiera ramkę danych i zwraca kolumnę wypełnioną wartościami oddzielonymi przecinkami poszczególnych wartości w ramce danych.

In [ ]:
def get_dummies_col(df, sep=","):

    ## zainicjuj pustą listę do przechowywania kolumny stringów, która zostanie użyta do utworzenia fikcyjnej ramki danych
    dummies_col = []
    
    ## iteruj po każdym wierszu w ramce danych w sposób, w który możemy uzyskać dostęp do poszczególnych elementów komórek
    ## uzyskaj listę wartości komórek z wiersza (np. listę języków programowania). Upewnij się, że nie dodajesz NaNs
    ## połącz je jako ciąg oddzielony przecinkami i dołącz go do pustej listy w kolumnie
    for row in df.itertuples(index=False):
        values = [item for item in row if isinstance(item, str)]
        dummies_col.append(sep.join(values))
        
    ## utwórz nową kolumnę w dataframe o nazwie "dummies", która przyjmuje zawartość kolumny dummies
    df["dummies"] = dummies_col
    
    ## zamień wszystkie "" i "None" z ramki danych na "None/NA"
    values_to_update = {"dummies": {"": "None/NA", "None": "None/NA"}}
    df = df.replace(values_to_update)
    
    ## zwróć nową ramkę
    return df

In [ ]:
## Stwórz ramkę dla frameworków metodą dummies
framework_dummies = framework_df['frameworks_used'].str.get_dummies(sep=',')
framework_dummies

Stwórz ogólną funkcję `dummies_from_series()`, która pobiera ramkę danych i argument separatora i zwraca dummies dla Serii

In [ ]:
def dummies_from_series(series, sep=","):
    ## zwraca dummy ramkę danych z ramki danych.
    # Zapamiętaj, która kolumna została użyta do przypisania ciągów, nad którymi chcemy utworzyć dummies
    return series["dummies"].str.get_dummies(sep=sep)

In [ ]:
## Utwórz nową ramkę danych, która połączy tę ramkę danych z rolami stanowisk
frameworks_for_job_role = framework_dummies.join(df["Q5"]).rename(columns={"Q5": "Job Title"})
frameworks_for_job_role

In [ ]:
## Pogrupuj ramkę danych według tytułu stanowiska i zagreguj według języków programowania
frameworks_for_job_role_grouped = frameworks_for_job_role.groupby(["Job Title"]).sum()
frameworks_for_job_role_grouped

Utwórz ogólną funkcję `group_dummies_by()`, która pobiera ramkę danych dummies oraz Serię i zwraca dummies pogrupowane i zagregowane według Serii.

In [ ]:
def group_dummies_by(dummies_df, series):
    series_name = series.name
    to_group = dummies_df.join(series)
    grouped = to_group.groupby([series_name]).sum()
    
    return grouped

In [ ]:
framework_df = get_df_for_dummies(df, "Q28_Part_", 13)
framework_df = get_dummies_col(framework_df)
framework_dummies = dummies_from_series(framework_df)
frameworks_for_job_role_grouped = group_dummies_by(framework_dummies, df["Q5"])
frameworks_for_job_role_grouped = frameworks_for_job_role_grouped.div(
    frameworks_for_job_role_grouped.sum(axis=1), axis=0)
frameworks_for_job_role_grouped.index.rename("Job Role", inplace=True)
frameworks_for_job_role_grouped

In [ ]:
## Utwórz mapę cieplną powyższej ramki danych
px.imshow(frameworks_for_job_role_grouped, title="Heatmap of preferred Frameworks per Job Role")

**Obserwacje**:
- None/NA może wprowadzać w błąd. Nie zawsze oznacza to, że ich rola nie wymaga frameworków, ponieważ obejmuje również osoby, które być może nie odpowiedziały na pytanie, ponieważ ich narzędzie użytkowania nie było dostępne jako opcja. Na przykład statystyk może używać frameworka R, którego opcja nie jest tutaj podana.
- Scikit-learn jest najpopularniejszym wymienionym narzędziem
- Statystycy często używają lasu losowego (Random Forest)

**Roczne wynagrodzenie**

Zacznijmy teraz rozważać naszą zmienną docelową! Zaczniemy od od początku. Narysujmy histogram zarobków i posortujmy ten wykres w kolejności zakresów wynagrodzeń

In [ ]:
px.histogram(df, "Q10", labels={"Q10": "Salary"}, title="Count of salary ranges (Unsorted)")

Widzę już z tego kilka interesujących ustaleń, ale zanim wspomnę o czymkolwiek, chcę, abyśmy posortowali oś x w kolejności wartości liczbowej. Oznacza to, że skrajna lewa kolumna to `0-999 USD`, a skrajna prawa to `> 500 000 USD`.

Poświęć trochę czasu na przemyślenie i wdrożenie sposobu rozwiązania tego problemu. Jeśli po kilku minutach nie jesteś w stanie wymyślić/wymyślić rozwiązania, przeczytaj i zaimplementuj wg. poniższych kroków.

<details>
<summary><b> Kliknij tutaj, aby wyświetlić moje rozwiązanie</b></summary>
<ul>
<li>Utwórz nową ramkę danych z samymi pensjami</li>
<li>Utwórz set wynagrodzeń (unikalne wartości)</li>
<li>Utwórz mapowanie kategorii wynagrodzeń na int, gdzie int to pierwsza numeryczna część ciągu (np. <code>{"$0-999": 0, "100,000-124,999": 100000}</code >). Ta część będzie wymagała użycia metod `.replace()` i `.split()` natywnych dla Pythona
<li>Zastąp pensje w ramce danych wartością całkowitą</li>
<li>Posortuj ramkę danych w rosnącej kolejności numerycznej</li>
<li>Odwróć mapowanie i zastąp int ich wariantem ciągu</li>
<li>Wykreśl ramkę danych i zastąp etykiety x ciągami wynagrodzeń</li>
</ul>
</details>

In [ ]:
salary_df = pd.DataFrame(df["Q10"])
salary_df.rename(columns={"Q10": "Salary"}, inplace=True)
salary_set = set(salary_df["Salary"])
salary_string_int_dict = dict()

for string_salary in salary_set:
    
    if isinstance(string_salary, float): continue
        
    salary = string_salary.replace("$", "").replace("> ", "").replace(",", "")
    salary = salary.split("-")[0]
    salary_string_int_dict[string_salary] = int(salary)

values_to_update = {"Salary": salary_string_int_dict}
salary_df = salary_df.replace(values_to_update)
salary_df = salary_df.sort_values("Salary")

salary_int_string_dict = {v:k for k,v in salary_string_int_dict.items()}
values_to_update = {"Salary": salary_int_string_dict}
salary_df = salary_df.replace(values_to_update)

percent_na = np.round(100 * salary_df["Salary"].isna().sum()/len(salary_df), 2)
print("Percent of users who didn't answer the salary question:", percent_na)
px.histogram(salary_df, "Salary", title="Count of Salary ranges")

Prawie 37% uczestników ankiety wprost nie odpowiedziało na to pytanie. Wydaje się, że jest dziwnie duża liczba ludzi, którzy zarabiają od \\$0 do \\$999 rocznie. Podejrzewam, że jest to tak wysokie, ponieważ wiele osób, które nie chciały odpowiedzieć na pytanie (i nie zdawały sobie sprawy, że jest to opcjonalne) zaznaczyło to pole. Możemy również zauważyć kilka innych interesujących faktów - to znaczy - wydaje się, że mamy "dwa" szczyty przy bardzo różnych zarobkach - jeden na poziomie 10 000 - 14 999, a drugi na 100 000 - 124 999. Czy możesz pomyśleć, dlaczego tak może być? Wygląda też na to, że jest kilku bardzo bogatych kagglerów, z których 83 zarabia ponad $500k rocznie.

Trzy najbardziej popularne grupy zarobków (poza 0 - 999) wydają się wynosić 10 000 - 14 999, 100 000 - 124 999 i 30 000 - 39 999. Przygotuj kartogram przedstawiający medianę wynagrodzeń w krajach, abyśmy mogli lepiej rozróżnić, w jakich regionach możemy spodziewać się zarobków

In [ ]:
## Przygotuj kartogram przedstawiający medianę wynagrodzeń w krajach
median_salaries_df = df[["Q3", "Q10"]]
median_salaries_df.rename(columns={"Q3": "Country", "Q10": "Salary"}, inplace=True)
values_to_update = {"Salary": salary_string_int_dict}
median_salaries_df = median_salaries_df.replace(values_to_update)
median_salaries_df = median_salaries_df.groupby(["Country"]).median()
median_salaries_df

In [ ]:
country_codes = []
for country in median_salaries_df.index:
    country_code = pycountry.countries.search_fuzzy(country)[0] # Weź pierwszy element zwracany przez przybliżone wyszukiwanie
    country_codes.append(country_code.alpha_3)

median_salaries_df["Country Code"] = country_codes
median_salaries_df

In [ ]:
salaries_series = median_salaries_df["Salary"]
values_to_update = {"Salary": salary_int_string_dict}
median_salaries_df = median_salaries_df.replace(values_to_update)
median_salaries_df["Salary Values"] = salaries_series

In [ ]:
px.choropleth(median_salaries_df, locations="Country Code", hover_name=median_salaries_df.index, color="Salary Values", hover_data=["Salary"], title="Median Salaries by Country")

Biorąc pod uwagę liczbę uczestników z USA i Indii, którą rozpoznaliśmy wcześniej, możemy przyjąć te wartości, aby były bardziej poprawne w stosunku do rozkładu generowania danych niż w większości innych krajów. Działając przy tym założeniu, wygląda na to, że w USA i Szwajcarii rozsądnie byłoby być na 100 000+, podczas gdy w Indiach najprawdopodobniej zarabialiby coś w przedziale 7500 - 9999. Wygląda na to, że Australia też ma kilka dobrze płatnych miejsc pracy. Spodziewalibyśmy się, że średnia pensja w Wielkiej Brytanii będzie wyższa niż 10 000 - 14 999. Jakie rzeczy moglibyśmy zbadać, aby określić, dlaczego ta wartość wynagrodzenia jest niższa, niż się spodziewamy?

A co z wynagrodzeniami według płci?

In [ ]:
salaries_by_gender_df = df[["Q2", "Q10"]]
salaries_by_gender_df.rename(columns={"Q2": "Gender", "Q10": "Salary"}, inplace=True)
values_to_update = {"Salary": salary_string_int_dict}
salaries_by_gender_df = salaries_by_gender_df.replace(values_to_update)
salaries_by_gender_df

Stwórzmy wykres pudełkowy salary per gender

Chociaż wygląda na to, że którakolwiek z kategorii jest w stanie osiągnąć najwyższy przedział zarobków, wydaje się, że kobiety mają najniższą medianę zarobków. Wydaje się, że ci, którzy wolą opisywać siebie, mają również wyższe średnie zarobki.

Jakie są najlepiej płatne prace? Wykreślmy średnią pensję każdej roli

In [ ]:
salaries_job_df = df[["Q5", "Q10"]]
salaries_job_df.rename(columns={"Q5": "Job Title", "Q10": "Salary"}, inplace=True)

values_to_update = {"Salary": salary_string_int_dict}
salaries_job_df = salaries_job_df.replace(values_to_update)
salaries_job_df

# salary_series = salaries_job_df["Salary"]
# salaries_job_df["Salary Bracket"] = salary_series
# salaries_job_df

In [ ]:
grouped_mean_salaries = salaries_job_df.groupby(["Job Title"]).mean().reset_index().sort_values(by="Salary", ascending=False)
grouped_mean_salaries.dropna(inplace=True)
px.bar(grouped_mean_salaries, "Job Title", "Salary", labels={"Salary": "Mean Salary"}, title="Mean Salary per Job Role")

Jaki może być może bardziej informacyjny sposób przedstawienia tych wynagrodzeń? Myślę o wykresie pudełkowym. Odrzućmy pensje NA i „0” i zwizualizujmy to

### ZADANIE 6 (1 pkt):
Stwórz wykres pudełkowy dla wynagrodzeń.

In [ ]:
salaries_job_df.dropna(inplace=True)
salaries_job_df = salaries_job_df[(salaries_job_df["Salary"] != 0)]


fig.show()

**Obserwacje**:
- Ankieta wydaje się wskazywać, że wszystkie role zawodowe mają potencjał do zdobycia pracy za \\$500k+ poza pracą inżyniera bazy danych
- Na całym świecie inżynierowie oprogramowania i analitycy danych mają dwie najniższe średnie pensje - przy czym inżynier oprogramowania ma niższą medianę wynagrodzenia niż analityk danych, ale wyższą średnią (jak widać na pierwszym wykresie)
- Zarządzanie produktami/projektami i Data Science 😉 wydają się być najbardziej lukratywnymi stanowiskami pracy - przy czym ta pierwsza nieznacznie przejmuje inicjatywę

Zastanówmy sie, jaki jest procent data scientists, którzy zarabiają powyżej \\500 tys. USD w porównaniu z kierownikami projektów:

In [ ]:
num_data_scientists_above_500 = len(salaries_job_df[(salaries_job_df["Job Title"] == "Data Scientist") & (salaries_job_df["Salary"] == 500000)])
num_project_managers_above_500 = len(salaries_job_df[(salaries_job_df["Job Title"] == "Product/Project Manager") & (salaries_job_df["Salary"] == 500000)])

percent_ds_above_500 = 100 * num_data_scientists_above_500/len(salaries_job_df)
percent_pm_above_500 = 100 * num_project_managers_above_500/len(salaries_job_df)

print("The percent of Data Scientists who earn above $500,000: {}%".format(np.round(percent_ds_above_500, 2)))
print("The percent of Project Managers who earn above $500,000: {}%".format(np.round(percent_pm_above_500, 2)))

Zobaczmy, jaki wpływ mają lata programowania (Q11) na wynagrodzenia

In [ ]:
programming_experience_salary_df = df[["Q10", "Q15"]]
programming_experience_salary_df.rename(columns={"Q10": "Salary", "Q15": "Programming Experience"}, inplace=True)
values_to_update = {"Salary": salary_string_int_dict}
programming_experience_salary_df = programming_experience_salary_df.replace(values_to_update)
programming_experience_salary_df

In [ ]:
category_array = ["I have never written code", "< 1 years", "1-2 years", "3-5 years", "5-10 years", "10-20 years", "20+ years"]
# fig = px.scatter(programming_experience_salary_df, "Programming Experience", "Salary", title="Density of Programming Experience vs Salary")
fig = px.scatter(programming_experience_salary_df, "Programming Experience", "Salary", facet_col=df["Q2"],title="Density of Programming Experience vs Salary")
fig.update_traces(marker=dict(
            opacity=0.05,
            size=20,
            line=dict(
                color='MediumPurple',
                width=0.5
            )))
fig.update_layout(xaxis={'categoryorder':'array', 'categoryarray':category_array})
fig.show()

**Obserwacje**:
- Są ludzie zarabiający \\$500k+, którzy nigdy nie napisali kodu. To samo z ludźmi, którzy mają mniej niż rok doświadczenia w programowaniu.
- Istnieje jednak trend, w którym bardziej doświadczeni programiści zarabiają wyższe wynagrodzenie na wszystkich poziomach wynagrodzeń
- Nie ma zbyt wielu doświadczonych kobiet (tj. ponad 10-20 lat doświadczenia)

Teraz narysujemy kolejny diagram Sankeya – tym razem śledzący, w jaki sposób płeć, wiek, stopień naukowy, rola i kraj wpływają na twoją pensję. Aby to osiągnąć, uogólnij kod, który napisałam wcześniej dla diagramu Sankeya, do funkcji `get_sankey_data`, która jako argument pobiera ponownie zindeksowaną ramkę danych i zwraca node_labels oraz słownik `link`. Następnie użyj tych elementów, aby wykreślić diagram Sankeya

### *ZADANIE 7 (10 pkt):
Zadanie dla chętnych, na dodatkowe punkty
Stwórzy wykres z podwykresami, który będzie pokazywał Percent of Known Programming Languages by Age
Teraz narysujemy kolejny diagram Sankeya – tym razem śledzący, w jaki sposób płeć, wiek, stopień naukowy, rola i kraj wpływają na twoją pensję. Aby to osiągnąć, uogólnij kod, który napisałam wcześniej dla diagramu Sankeya, do funkcji `get_sankey_data`, która jako argument pobiera ponownie zindeksowaną ramkę danych i zwraca node_labels oraz słownik `link`. Następnie użyj tych elementów, aby wykreślić diagram Sankeya

In [ ]:
## Wykorzystaj ponownie kod do Sankeya


In [ ]:
## utwórz nową ramkę danych z odpowiednimi zmiennymi, za pomocą których chcemy wykreślić naszego Sankeya, ponownie zindeksuj go i przekaż do funkcji get_sankey_data


In [ ]:


fig.update_layout(title_text="Sankey Diagram (Gender, Age, Education, Role, Country, Salary)", font_size=10, height=1000)
fig.show()

Powyższy schemat jest chaotyczny, uporządkujmy go w następujący sposób:
- Pogrupuj kraje według kontynentów (znowu nie zmienimy Indii i USA)
- Stwórz szersze przedziały na pensję (może 5 przedziałów?), usuwając \\$0-999
- Dodaj doświadczenie w programowaniu jako kolumnę na powyższym wykresie

In [ ]:
## Zamień kraje na kontynenty
values_to_update = {"Country": countries_continent_dict}
salaries_sankey_df = salaries_sankey_df.replace(values_to_update)
salaries_sankey_df

,Gender,Age,Education,Role,Country,Salary
11006,Male,18-21,Bachelor’s degree,Student,Asia,NaN
10555,Male,18-21,Master’s degree,Student,India,NaN
10551,Male,18-21,Bachelor’s degree,Student,Asia,NaN
19073,Male,18-21,Master’s degree,Data Scientist,United States of America,NaN
10550,Male,18-21,Bachelor’s degree,Software Engineer,India,"1,000-1,999"
...,...,...,...,...,...,...
6790,Male,70+,Some college/university study without earning ...,Not employed,North America,NaN
2057,Male,70+,Doctoral degree,Research Scientist,United States of America,"300,000-500,000"
8874,Male,70+,Doctoral degree,Other,United States of America,"100,000-124,999"
11743,Prefer not to say,70+,I prefer not to answer,Other,Other,NaN


In [ ]:
## Usuń wiersze z pensją $0-999 oraz zwiększ przedziały pensji
set(salaries_sankey_df["Salary"])
salaries_sankey_df = salaries_sankey_df[salaries_sankey_df["Salary"] != "$0-999"]
under_30000, under_80000, under_150000, under_300000, under_500000 = "0 - 29,999", "30,000 - 79,999", "80,000 - 149,999", "150,000 - 299,999", "300,000 - 500,000" 
salary_wider_bins_dict = dict()
for salary_string in set(salaries_sankey_df["Salary"]):
    
    if salary_string == "> $500,000" or isinstance(salary_string, float):
        continue
    
    salary_upper_bound = salary_string.split("-")[-1]
    salary_upper_bound = salary_upper_bound.replace(",", "")
    salary_upper_bound = int(salary_upper_bound)
    
    if salary_upper_bound < 30000:
        salary_wider_bins_dict[salary_string] = under_30000
    elif salary_upper_bound < 80000:
        salary_wider_bins_dict[salary_string] = under_80000
    elif salary_upper_bound < 150000:
        salary_wider_bins_dict[salary_string] = under_150000
    elif salary_upper_bound < 300000:
        salary_wider_bins_dict[salary_string] = under_300000
    elif salary_upper_bound < 500000:
        salary_wider_bins_dict[salary_string] = under_500000

values_to_update = {"Salary": salary_wider_bins_dict}
salaries_sankey_df = salaries_sankey_df.replace(values_to_update)
salaries_sankey_df

,Gender,Age,Education,Role,Country,Salary
11006,Male,18-21,Bachelor’s degree,Student,Asia,NaN
10555,Male,18-21,Master’s degree,Student,India,NaN
10551,Male,18-21,Bachelor’s degree,Student,Asia,NaN
19073,Male,18-21,Master’s degree,Data Scientist,United States of America,NaN
10550,Male,18-21,Bachelor’s degree,Software Engineer,India,"0 - 29,999"
...,...,...,...,...,...,...
6790,Male,70+,Some college/university study without earning ...,Not employed,North America,NaN
2057,Male,70+,Doctoral degree,Research Scientist,United States of America,"300,000-500,000"
8874,Male,70+,Doctoral degree,Other,United States of America,"80,000 - 149,999"
11743,Prefer not to say,70+,I prefer not to answer,Other,Other,NaN


In [ ]:
## Utwórz nową kolumnę w salaries_sankey_df, która jest długością doświadczenia w programowaniu
salaries_sankey_df["Programming Experience"] = df["Q15"]
salaries_sankey_df

,Gender,Age,Education,Role,Country,Salary,Programming Experience
11006,Male,18-21,Bachelor’s degree,Student,Asia,NaN,< 1 years
10555,Male,18-21,Master’s degree,Student,India,NaN,< 1 years
10551,Male,18-21,Bachelor’s degree,Student,Asia,NaN,< 1 years
19073,Male,18-21,Master’s degree,Data Scientist,United States of America,NaN,NaN
10550,Male,18-21,Bachelor’s degree,Software Engineer,India,"0 - 29,999",3-5 years
...,...,...,...,...,...,...,...
6790,Male,70+,Some college/university study without earning ...,Not employed,North America,NaN,< 1 years
2057,Male,70+,Doctoral degree,Research Scientist,United States of America,"300,000-500,000",20+ years
8874,Male,70+,Doctoral degree,Other,United States of America,"80,000 - 149,999",1-2 years
11743,Prefer not to say,70+,I prefer not to answer,Other,Other,NaN,NaN


In [ ]:
## Zmień indeks i wy świetl wykres Sankeya
# Usuń kolumny Age i Gender ci zmień indeksy ramki na:
# Role, Programming Experience, Region, Education, Salary
salaries_sankey_df = salaries_sankey_df.rename(columns={"Country": "Region"})
salaries_sankey_df = salaries_sankey_df.reindex(["Role", "Programming Experience", "Region", "Education", "Salary"], axis=1)
node_labels, link = get_sankey_data(salaries_sankey_df)
fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = node_labels,
      color = "blue"
    ),
    link = link)])

fig.update_layout(title_text="Sankey Diagram (Role, Programming Experience, Region, Education, Salary)", font_size=10, height=800)
fig.show()

Informacje i ścieżki do różnych przedziałów wynagrodzeń stają się teraz znacznie bardziej zrozumiałe do interpretacji. Niektóre interesujące odkrycia, które znalazłam na tym wykresie, to fakt, że jest kilku inżynierów oprogramowania i naukowców zajmujących się danymi, którzy nigdy wcześniej nie pisali kodu - nie jestem przekonana, czy to prawda, ale jest ich niewielu, więc moglibyśmy to przyjąć jako obserwacje odstające. Większość osób zarabiających powyżej 150 000 wydaje się mieć stopień magistra lub doktora. Zabawa z kolejnością indeksowania może szybko umożliwić wyciągnięcie wniosków z różnych kombinacji kolumn (np. umieszczając Education przed Job title, możemy zobaczyć, w jakie role obejmą ludzie, na podstawie ich wykształcenia).

Ostatni wykres dla pytania 9, które dotyczy umiejętności i obowiązków związanych z ich obecną pracą. Utwórz poboczne wykresy tych umiejętności, zagregowane, dla każdego przedziału wynagrodzeń. Użyj 6 przedziałów wynagrodzeń, które zdefiniowaliśmy wcześniej. Upewnij się, że kolumny tworzonej ramki danych skills są nienaruszone i zademonstruj swój wykres/i wszelkie inne dane wyjściowe Pythona w zrozumiały sposób. 

In [ ]:
skills_df = get_df_for_dummies(df, "Q9_Part_", 9)
skills_df = get_dummies_col(skills_df, sep="::")
skills_dummies = dummies_from_series(skills_df, sep="::")
skills_grouped = group_dummies_by(skills_dummies, salaries_sankey_df["Salary"])
skills_grouped = skills_grouped.reindex(["0 - 29,999", "30,000 - 79,999", "80,000 - 149,999", "150,000 - 299,999", "300,000-500,000", "> $500,000"])
skills_grouped

,Analyze and understand data to influence product or business decisions,Build and/or run a machine learning service that operationally improves my product or workflows,"Build and/or run the data infrastructure that my business uses for storing, analyzing, and operationalizing data",Build prototypes to explore applying machine learning to new areas,Do research that advances the state of the art of machine learning,Experimentation and iteration to improve existing ML models,None of these activities are an important part of my role at work,None/NA,Other
Salary,,,,,,,,,
"0 - 29,999",2030,1094,1140,1497,825,1143,240,1592,91
"30,000 - 79,999",1674,901,984,1441,585,1010,138,775,62
"80,000 - 149,999",1237,705,767,1121,397,772,74,317,41
"150,000 - 299,999",465,278,281,446,194,368,18,62,12
"300,000-500,000",46,33,36,47,29,41,5,8,1
"> $500,000",41,24,30,30,22,29,2,24,4


In [ ]:
skills_df_columns = skills_grouped.columns
skills_df_columns_mapping = {col: i for i, col in enumerate(skills_df_columns)}
skills_grouped = skills_grouped.rename(columns=skills_df_columns_mapping)
[print(i, "\t", col) for col, i in skills_df_columns_mapping.items()]
print()

0 	 Analyze and understand data to influence product or business decisions
1 	 Build and/or run a machine learning service that operationally improves my product or workflows
2 	 Build and/or run the data infrastructure that my business uses for storing, analyzing, and operationalizing data
3 	 Build prototypes to explore applying machine learning to new areas
4 	 Do research that advances the state of the art of machine learning
5 	 Experimentation and iteration to improve existing ML models
6 	 None of these activities are an important part of my role at work
7 	 None/NA
8 	 Other



In [ ]:
## Stwórz wykresy. Oś x powinna być 4 najlepszymi językami programowania, a oś y ich liczbą.
fig = make_subplots(2, 3, subplot_titles=skills_grouped.index)
for i, role in enumerate(skills_grouped.index):
    row = (i // 3) + 1
    col = (i % 3) + 1
    skills_values = skills_grouped.iloc[i]
    fig.add_trace(
        go.Bar(x=skills_values.index, y=skills_values),
        row=row, col=col
    )
    fig.update_xaxes(type="category")
fig.update_layout(showlegend=False, height=800, title="Skills/Responsibilities per Salary Bracket")
fig.show()

**Obserwacje:**
- We wszystkich przedziałach wynagrodzeń obecne jest 0 -„Analyze and understand data to influence product or business decisions”.
- Oprócz najniższego i najwyższego przedziału, wygląda na to, że wiele zadań obejmuje 3 - "Builing prototypes which explores applying machine learning to new area".
- Dwa górne przedziały wynagrodzeń mają wyższą względną częstotliwość odpowiedzi 4 - "Do research that advances the state of the art of machine learning".
- Wydaje się, że w ostatnich trzech nawiasach wyższy odsetek pracowników ma odpowiedzialność za opcję 5 - "Experimentation and iteration to improve existing ML models".
- Opcja 7 - "None/NA" - wydaje się być bardziej rozpowszechniona w dwóch dolnych nawiasach i najwyższym nawiasie. Podejrzewam, że powodem tego jest to, że żaden z opisów stanowisk nie odnosi się do tych osób, podczas gdy pozostałe trzy przedziały mają obowiązki skoncentrowane bardziej na data science i uczeniu maszynowym.

Jaki rodzaj wykresu mógł pokazać te informacje w bardziej przejrzysty i zwięzły sposób?